In [1]:
%pip install --upgrade transformers peft regex bitsandbytes


   ---------------------------------------- 0.0/72.2 MB ? eta -:--:--
   - -------------------------------------- 2.4/72.2 MB 15.0 MB/s eta 0:00:05
   -- ------------------------------------- 4.2/72.2 MB 12.0 MB/s eta 0:00:06
   --- ------------------------------------ 5.8/72.2 MB 10.1 MB/s eta 0:00:07
   ---- ----------------------------------- 7.6/72.2 MB 9.6 MB/s eta 0:00:07
   ----- ---------------------------------- 10.2/72.2 MB 10.3 MB/s eta 0:00:07
   ------ --------------------------------- 12.6/72.2 MB 10.5 MB/s eta 0:00:06
   -------- ------------------------------- 14.9/72.2 MB 10.6 MB/s eta 0:00:06
   --------- ------------------------------ 17.3/72.2 MB 10.6 MB/s eta 0:00:06
   ---------- ----------------------------- 19.7/72.2 MB 10.6 MB/s eta 0:00:05
   ------------ --------------------------- 22.0/72.2 MB 10.8 MB/s eta 0:00:05
   ------------- -------------------------- 24.6/72.2 MB 11.0 MB/s eta 0:00:05
   --------------- ------------------------ 28.0/72.2 MB 11.4 MB/s

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"  # or path to your base model
LORA_PATH = "../mlruns/232166607607031798/7aceb521194d4286be3e485efc1272f8/artifacts/adapters/epoch-6"                   # path to your LoRA folder
MERGED_PATH = f"../mlruns/232166607607031798/7aceb521194d4286be3e485efc1272f8/artifacts/adapters/epoch-6/merged"                 # where merged model will be saved


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, 
                                                    # quantization_config=quantization_config , 
                                                    torch_dtype=torch.float16,
                                                    device_map="cpu")
    
model = PeftModel.from_pretrained(base_model, LORA_PATH)
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)

print(type(model))

model = model.merge_and_unload()  # merges LoRA weights into base weights

print(type(model))

model = model.half()

# Save merged model in Hugging Face format
model.save_pretrained(MERGED_PATH, safe_serialization=True)
tokenizer.save_pretrained(MERGED_PATH)

print(f"Merged model saved to {MERGED_PATH}")


c:\Users\Admin\miniconda3\envs\gguf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.83s/it]
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


<class 'peft.peft_model.PeftModel'>
<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>
Merged model saved to ../mlruns/232166607607031798/7aceb521194d4286be3e485efc1272f8/artifacts/adapters/epoch-6/merged


In [3]:
# python .\convert_hf_to_gguf.py C:\Users\Admin\Documents\Projects\Repositories\synthetic-data-generator\mlruns\232166607607031798\7aceb521194d4286be3e485efc1272f8\artifacts\adapters\epoch-6\merged --outfile C:\Users\Admin\Documents\Projects\Repositories\synthetic-data-generator\mlruns\232166607607031798\7aceb521194d4286be3e485efc1272f8\artifacts\adapters\epoch-6\merged\mistral7b-merged-f16.gguf --outtype f16